In [1]:
import sys
sys.path.insert(0, "/home/mickaelz/Recommendation systems/project")
# from data_generation import generate

In [2]:
import os
from data_generation import generate
from model_training import training
from model_evaluation import evaluation
from clustering import train_kmeans, train_inputfeatureskmeans
from options import config
from fuzzy_clustering import train_fuzzy_kmeans
from evaluation_fuzzy_only_on_cold import evaluation_with_fuzzy_on_cold
import warnings

import time
import pandas as pd
import torch


In [8]:
import torch
# prnt cuda device if there is
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [3]:

try:
    pd.io.parquet.get_engine('fastparquet')
except ImportError as e:
    print("please install fastparquet first")

if config['use_cuda'] and not torch.cuda.is_available():
    print("please make cuda gpu available, or set use_cuda=False")
    assert torch.cuda.is_available()

master_path= "/home/mickaelz/Recommendation systems/project/deezer"
dataset_path = "/home/mickaelz/Recommendation systems/project/data"
embeddings_version = config["embeddings_version"]
model_filename = "regression_model_" + embeddings_version
clustering_path = "clustering_" + embeddings_version
clusters_filename = "clustering_model" + embeddings_version
fuzzy_clustering_path = "fuzzy_clustering_" + embeddings_version
fuzzy_clusters_filename = "fuzzy_clustering_model" + embeddings_version
inputfeaturesclustering_path = "inputfeaturesclustering_" + embeddings_version
inputfeaturesclusters_filename = "inputfeaturesclustering_model" + embeddings_version
print("--- running for embeddings version " + embeddings_version + " ---")

--- running for embeddings version svd ---


## Generating Data
### The function "generate" reads song and user embeddings and features from parquet files, constructs train/validation/test splits, and saves per-user feature and label pairs as pickled PyTorch tensors for downstream recommendation model training and evaluation.

In [ ]:
if not os.path.exists("{}/".format(master_path)):
    os.mkdir("{}/".format(master_path))
if not os.path.exists(master_path + "/" + embeddings_version + "/"):
    print("--- the data has not been generated yet for the embeddings version " + embeddings_version + " : generation running ---")
    os.mkdir(master_path + "/" + embeddings_version + "/")
    # preparing dataset
    print("--- data generation ---")
    start_time_data_generation = time.time()

    generate(dataset_path, master_path, config['embeddings_version'])
    
    print("--- data generation done ---")
    print("--- seconds ---" + str(time.time() - start_time_data_generation))
else:
    print("--- the data has already been generated : no need to regenerate it ---")

--- the data has not been generated yet for the embeddings version svd : generation running ---
--- data generation ---
--- songs data ---


FileNotFoundError: [Errno 2] No such file or directory: '/sise/home/mickaelz/data/song_embeddings.parquet'

In [11]:
''' training model. '''
print("--- training prediction model ---")
start_time_prediction_model = time.time()
training(dataset_path, master_path, embeddings_version = embeddings_version, save_model = True, model_filename = model_filename)
print("--- training prediction model done ---")
print("--- seconds ---" + str(time.time() - start_time_prediction_model))

--- training prediction model ---
Master path: /home/mickaelz/Recommendation systems/project/deezer
⋯ Building shard /home/mickaelz/Recommendation systems/project/deezer/svd/train/svd_train_shard.pt
⋯ Loading shard /home/mickaelz/Recommendation systems/project/deezer/svd/train/svd_train_shard.pt
⋯ Building shard /home/mickaelz/Recommendation systems/project/deezer/svd/validation/svd_validation_shard.pt
⋯ Loading shard /home/mickaelz/Recommendation systems/project/deezer/svd/validation/svd_validation_shard.pt
Epoch 00  train‑MSE 0.01013  [45.9s]
Epoch 01  train‑MSE 0.00574  [0.6s]
Epoch 02  train‑MSE 0.00452  [0.6s]
Epoch 03  train‑MSE 0.00399  [0.6s]
Epoch 04  train‑MSE 0.00372  [0.6s]
Epoch 05  train‑MSE 0.00356  [0.6s]
Epoch 06  train‑MSE 0.00343  [0.6s]
Epoch 07  train‑MSE 0.00331  [0.6s]
Epoch 08  train‑MSE 0.00326  [0.6s]
Epoch 09  train‑MSE 0.00318  [0.6s]
  val‑loss 273357850.80000
Epoch 10  train‑MSE 0.00315  [0.6s]
Epoch 11  train‑MSE 0.00313  [0.6s]
Epoch 12  train‑MSE 0.0030

In [ ]:
'''Hard clustering'''
warnings.filterwarnings("ignore")

# evaluation of the model - semi personalization strategy.
if not os.path.exists("{}/{}/".format(master_path, clustering_path)):
    os.mkdir("{}/{}/".format(master_path, clustering_path))
if not os.path.exists(master_path + "/" + clustering_path + "/" + clusters_filename):
    print("--- clustering running ---")
    start_time_clustering = time.time()
    train_kmeans(dataset_path, master_path, clustering_path, config['nb_clusters'], config['max_iter'], config['random_state'], embeddings_version = embeddings_version, clusters_filename = clusters_filename)
    print("--- clustering done ---")
    print("--- seconds ---" + str(time.time() - start_time_clustering))
else:
    print("--- no need to do the clustering again ---")

--- clustering running ---


/home/mickaelz/Recommendation systems/project/clustering.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user_embeddings[list_embeddings] = pd.DataFrame(user_embeddings[embeddings_version + "_embeddings"].tolist(),
/home/mickaelz/Recommendation systems/project/clustering.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user_embeddings[list_embeddings] = pd.DataFrame(user_embeddings[embeddings_version + "_embeddings"].tolist(),
/home/mickaelz/Recommendation systems/project/clustering.py:23: PerformanceWarning: DataFrame 

Enhanced cluster analysis saved to ./deezer/clustering_svd/clusters_analysis.png


/home/mickaelz/Recommendation systems/project/clustering.py:223: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user_embeddings["cluster"] = kmeans.labels_


Song probabilities computed for cluster: 0
Song probabilities computed for cluster: 100
Song probabilities computed for cluster: 200
Song probabilities computed for cluster: 300
Song probabilities computed for cluster: 400
Song probabilities computed for cluster: 500
Song probabilities computed for cluster: 600
Song probabilities computed for cluster: 700
Song probabilities computed for cluster: 800
Song probabilities computed for cluster: 900
--- clustering done ---
--- seconds ---172.8558042049408


## Baseline methods

### ***Input features clustering***
#### The “Input Features Clustering” baseline simply groups users with k-means on their raw, stacked demographic and registration-day interaction feature vectors (no neural network), then serves each cold user the 50 most-popular tracks from the warm-user cluster they’re assigned to

In [12]:

warnings.filterwarnings("ignore")

print("--- input features clustering baseline evaluation ---")
start_time_inputfeatures_clustering = time.time()
if not os.path.exists("{}/{}/".format(master_path, inputfeaturesclustering_path)):
    os.mkdir("{}/{}/".format(master_path, inputfeaturesclustering_path))
if not os.path.exists(master_path + "/" + inputfeaturesclustering_path + "/" + inputfeaturesclusters_filename):
    print("--- input features clustering running ---")
    start_time_inputfeaturesclustering = time.time()
    train_inputfeatureskmeans(dataset_path, master_path, inputfeaturesclustering_path, config['nb_clusters_inputfeatures'], config['max_iter'], config['random_state'], config["nb_songs"], embeddings_version = embeddings_version, clusters_filename = inputfeaturesclusters_filename)
    print("--- clustering done ---")
    print("--- seconds ---" + str(time.time() - start_time_inputfeatures_clustering))
else:
    print("--- no need to do the clustering again ---")

--- input features clustering baseline evaluation ---
--- no need to do the clustering again ---


In [14]:
warnings.filterwarnings("ignore")

print("--- input features clustering baseline evaluation ---")
start_time_inputfeaturesbaseline_eval = time.time()
evaluation(dataset_path, master_path, eval_type = "inputfeatures", embeddings_version = embeddings_version, model_filename = model_filename, clustering_path = inputfeaturesclustering_path, clusters_filename = inputfeaturesclusters_filename, nb_clusters = config['nb_clusters_inputfeatures'])
print("--- input features clustering baseline evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_inputfeaturesbaseline_eval))

--- input features clustering baseline evaluation ---
--- Load testing dataset ---
--- nb of test samples : 10000 ---
--- Load centroids for inputfeatures evaluation ---
--- nb of centroids : 100 ---
--- Load proba by segment for all song ids ---
--- nb of proba by segment for all song ids : 100 ---
--- Evaluation running : average precision, recall and ndcg ---
inputfeatures
eval done for 1000 users
eval done for 2000 users
eval done for 3000 users
eval done for 4000 users
eval done for 5000 users
eval done for 6000 users
eval done for 7000 users
eval done for 8000 users
eval done for 9000 users
length dataset : 10000
ndcg at 50 is : 0.09731791255401885
recall at 50 is : 0.029681496730659375
precision at 50 is : 0.0884119999999965
--- Evaluation running : standard deviation estimation ---
inputfeatures
length dataset : 10000
stddev ndcg at 50 is : 0.0024022154472845736
stddev recall at 50 is : 0.000805450610870239
stddev precision at 50 is : 0.002172657359088179
--- input features clu

### ***Popularity baseline***
#### The “Popularity” baseline serves each cold user the 50 most-popular tracks from the entire dataset, regardless of their demographic or registration-day interaction features


In [17]:

warnings.filterwarnings("ignore")

print("--- popularity baseline evaluation ---")
start_time_popbaseline_eval = time.time()
evaluation(dataset_path, master_path, eval_type = "popularity", embeddings_version = embeddings_version, model_filename = model_filename)
print("--- popularity baseline evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_popbaseline_eval))


--- popularity baseline evaluation ---
--- Load testing dataset ---
--- nb of test samples : 10000 ---
list of probabilities for each song for popularity baseline loaded
--- Evaluation running : average precision, recall and ndcg ---
popularity
eval done for 1000 users
eval done for 2000 users
eval done for 3000 users
eval done for 4000 users
eval done for 5000 users
eval done for 6000 users
eval done for 7000 users
eval done for 8000 users
eval done for 9000 users
length dataset : 10000
ndcg at 50 is : 0.09721713314529801
recall at 50 is : 0.030121982755237833
precision at 50 is : 0.0892219999999969
--- Evaluation running : standard deviation estimation ---
popularity
length dataset : 10000
stddev ndcg at 50 is : 0.0019764165631829525
stddev recall at 50 is : 0.0008004911395747795
stddev precision at 50 is : 0.0021198993686807944
--- popularity baseline evaluation done ---
--- seconds ---255.73762607574463


### ***avgd0stream baseline***
#### The avgd0stream baseline represents each cold user by averaging the embedding vectors of the tracks they streamed on registration day (day 0) and then recommends the 50 nearest tracks in that embedding space—falling back to a global-popularity list when the user streamed nothing

In [18]:

warnings.filterwarnings("ignore")

'''avg d0 stream baseline.'''
print("--- avg d0 stream baseline evaluation ---")
start_time_avgd0streambaseline_eval = time.time()
evaluation(dataset_path, master_path, eval_type = "avgd0stream", embeddings_version = embeddings_version, model_filename = model_filename)
print("--- avg d0 stream baseline evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_avgd0streambaseline_eval))



--- avg d0 stream baseline evaluation ---
--- Load testing dataset ---
--- nb of test samples : 10000 ---
--- Load song embeddings ---
--- nb of songs : 50000 ---
--- Evaluation running : average precision, recall and ndcg ---
avgd0stream
eval done for 1000 users
eval done for 2000 users
eval done for 3000 users
eval done for 4000 users
eval done for 5000 users
eval done for 6000 users
eval done for 7000 users
eval done for 8000 users
eval done for 9000 users
length dataset : 10000
ndcg at 50 is : 0.09714874501797381
recall at 50 is : 0.040537400220195356
precision at 50 is : 0.10876999999999527
--- Evaluation running : standard deviation estimation ---
avgd0stream
length dataset : 10000
stddev ndcg at 50 is : 0.0024461733996483295
stddev recall at 50 is : 0.0007976636664658093
stddev precision at 50 is : 0.002654174908412124
--- avg d0 stream baseline evaluation done ---
--- seconds ---294.5348582267761


## Fully perso

In [19]:

warnings.filterwarnings("ignore")

print("--- full personalisation evaluation ---")
start_time_fullperso_eval = time.time()
evaluation(dataset_path, master_path, eval_type = "full_perso", embeddings_version = embeddings_version, model_filename = model_filename)
print("--- full personalisation evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_fullperso_eval))

--- full personalisation evaluation ---
--- Load testing dataset ---
--- nb of test samples : 10000 ---
--- Load song embeddings ---
--- nb of songs : 50000 ---
--- Load model ---
RegressionTripleHidden(
  (dpin): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=2579, out_features=400, bias=True)
  (fc1_bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc2_bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=300, out_features=200, bias=True)
  (fc3_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=200, out_features=128, bias=True)
)
--- Evaluation running : average precision, recall and ndcg ---
full_perso
eval done for 1000 users
eval done for 2000 users
eval done for 3000 users
eval done for 4000 users
eval done for 5000 users
eval done for 6000 users
eva

## Semi perso

In [20]:
warnings.filterwarnings("ignore")

print("--- semi personalisation evaluation ---")
start_time_semiperso_eval = time.time()
evaluation(dataset_path, master_path, eval_type = "semi_perso", embeddings_version = embeddings_version, model_filename = model_filename, clustering_path=clustering_path, clusters_filename = clusters_filename, nb_clusters=config["nb_clusters"])
print("--- semi personalisation evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_semiperso_eval))

--- semi personalisation evaluation ---
--- Load testing dataset ---
--- nb of test samples : 10000 ---
--- Load song embeddings ---
--- nb of songs : 50000 ---
--- Load model ---
RegressionTripleHidden(
  (dpin): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=2579, out_features=400, bias=True)
  (fc1_bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc2_bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=300, out_features=200, bias=True)
  (fc3_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=200, out_features=128, bias=True)
)
--- Load centroids for semi perso evaluation ---
--- nb of centroids : 1000 ---
--- Load proba by segment for all song ids ---
--- nb of proba by segment for all song ids : 1000 ---
--- Evaluation running : average precision, re

## Our method

### Soft membership on cold users

In [4]:
warnings.filterwarnings("ignore")

print("--- soft membership on cold evaluation ---")
start_time_fuzzy_eval = time.time()
evaluation_with_fuzzy_on_cold(dataset_path, master_path, eval_type = "semi_perso", embeddings_version = embeddings_version, model_filename = model_filename, clustering_path=clustering_path, clusters_filename = clusters_filename, nb_clusters=config["nb_clusters"])
print("--- soft membership on cold evaluation done ---")
print("--- seconds ---" + str(time.time() - start_time_fuzzy_eval))

--- soft membership on cold evaluation ---
=== Starting Evaluation ===
--- Loading testing dataset from: /home/mickaelz/Recommendation systems/project/deezer/svd/test
Found 10000 test samples.
Loaded testing dataset.
Total test dataset size: 10000 samples.
Shape of first test sample: torch.Size([2579])
--- Loading song embeddings from: /home/mickaelz/Recommendation systems/project/data/song_embeddings.parquet
Loaded 50000 song embeddings.
--- Loading regression model from: /home/mickaelz/Recommendation systems/project/deezer/regression_model_svd.pt
Regression model loaded:
 RegressionTripleHidden(
  (dpin): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=2579, out_features=400, bias=True)
  (fc1_bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc2_bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=300, out_features=